## Foundation for script for automatic data generation 1H, 13C, HSQC, COSY NMR
- Master in notebook 20.1

In [ ]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles
from tqdm import tqdm
from rdkit import DataStructs
import json
import torch


def load_json_dics():
    with open('/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/itos.json', 'r') as f:
        itos = json.load(f)
    with open('/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/stoi.json', 'r') as f:
        stoi = json.load(f)

    with open('/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/stoi_MF.json', 'r') as f:
        stoi_MF = json.load(f)
    with open('/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/itos_MF.json', 'r') as f:
        itos_MF = json.load(f)    
    return itos, stoi, stoi_MF, itos_MF
    
itos, stoi, stoi_MF, itos_MF = load_json_dics()
rand_num = str(random.randint(1, 10000000))

In [ ]:
hyperparameters = {
    # General project information
    "project": ["Improv_Cycle_v1"],  # Name of the project for wandb monitoring
    "ran_num":[rand_num],
    #"random_seed":[42], # random_seed
    "device": ["cuda"], # device on which training takes place
    "gpu_num":[1], # number of GPUs for training with pytorch lightning
    "num_workers":[4], # Needs to stay 1 otherwise code crashes - ToDO
    "data_type":["sgnn"], #["sgnn", "exp", "acd", "real", "inference"], Different data types to select
    "execution_type":["validate_MMT"], #[ "plot_similarities", "simulate_real", "test_performance", "SMI_generation_MMT", "SMI_generation_MF", "data_generation", "transformer_training","transformer_improvement", "clip_training", "clip_improvement", "validate_MMT"] # different networks to select for training
    "syn_data_simulated": [False],  # For the improvment cycle a ticker that shows whether data has been simulated or not.
    "training_type":["clip"], #["clip","transformer"] # different networks to select for training

    # Encoding dicts
    "itos_path":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/itos.json"],
    "stoi_path":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/stoi.json"],
    "itos_MF_path":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/itos_MF.json"],
    "stoi_MF_path":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/stoi_MF.json"],
    
    ### Data settings
    "input_dim_1H":[2], # Imput dimensions of the 1H data
    "input_dim_13C": [1], # Imput dimensions of the 13C data
    "input_dim_HSQC": [2], # Imput dimensions of the HSQC data
    "input_dim_COSY": [2],  # Imput dimensions of the COSY data
    "input_dim_IR": [1000],  # Imput dimensions of the IR data
    "MF_vocab_size": [len(stoi_MF)],  # New, size of the vocabulary for molecular formulas
    "MS_vocab_size": [len(stoi)],  # New, size of the vocabulary for molecular formulas
    "tr_te_split":[0.9], # Train-Test split
    "padding_points_number":[64], # Padding number for the embedding layer into the network
    "data_size": [1000], # number of datapoints for the training 3975764/1797828
    "test_size": [10], # number of datapoints for the training 3975764
    "model_save_dir": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/test"], # Folder where networks are saved
    "ML_dump_folder": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/dump"], # a folder where intermediate files for the SGNN network are generated
    "model_save_interval": [10000], # seconds passed until next model is saved
    
    # Option 1 SGNN
    "use_real_data":[False], #[True, False]
    "ref_data_type":["1H"], #["1H","13C","HSQC","COSY","IR"]
    "csv_train_path": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_1H_comb_train_V8.csv'], # To keep a reference of the compounds that it was trained on
    "csv_1H_path_SGNN": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_1H_comb_train_V8.csv'],
    "csv_13C_path_SGNN": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_13C.csv'],    
    "csv_HSQC_path_SGNN": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_HSQC.csv'],    
    "csv_COSY_path_SGNN": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_COSY.csv'],      
   # "csv_IR_MF_path": [''],     #571124
    "csv_path_val": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_1H_comb_test_V8.csv'], #63459   
    #"IR_data_folder": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/IR_spectra_NN"],
    "IR_data_folder": [""],

   # "pickle_file_path": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_1H_comb_V8_938756.pkl"],
    "pickle_file_path": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/ML_NMR_5M_XL_1H_comb_test_V8_355655.pkl"],
    
    "dl_mode": ['val'], #["val","train"]   
    "isomericSmiles": [False], # whether stereochemistry is considered or not
    
    # Option 2 exp
    #"exp_path": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/9_ZINC_250k/missing_ZINC_files.csv'], #63459   

     # Option 2 ACD
    #"csv_path_1H_ACD": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/9_ZINC_250k/1H_ZINC_XL_v3.csv'],
    #"data_folder_HSQC_ACD": ["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/9_ZINC_250k/zinc250k"],
    # Option 3 real
    "comparision_number": [1000],  # With how many of the training examples should it be compared with in a t-SNE plot
    "vector_db": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/15_ZINC270M/smiles_fingerprints_train_4M_v1.csv'],    
    "secret_csv_SMI_vectors": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/25_Test_Improvement_cycle/1_Test_ZINC_250/test_32_zinc250_vec_db.csv'],    
    "secret_csv_SMI_targets": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/25_Test_Improvement_cycle/1_Test_ZINC_250/test_32_zinc250.csv'],    
    "secret_csv_SMI_sim_searched": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/25_Test_Improvement_cycle/1_Test_ZINC_250/test_32_zinc250.csv'],    
    "csv_SMI_targets": ['/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/25_Test_Improvement_cycle/1_Test_ZINC_250/test_32_zinc250_single_target_919.csv'],
    "csv_1H_path_REAL": [''],
    "csv_13C_path_REAL": [''],    
    "csv_HSQC_path_REAL": [''],    
    "csv_COSY_path_REAL": [''],    
    #"pkl_path_HSQC_real": [""],
    # noising HSQC data
    #"noising_HSQC":[False],
    #"noising_peaks_file":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v1/noise_peaks_norm_4.pkl"],
    #"noising_dist_file":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v1/noise_num_list_norm_4.pkl"],

    #### Transformer Settings ####
    # Training and model settings
    "training_mode":["1H_13C_HSQC_COSY_IR_MF_MW"], #["edding_src_1H = torch.zeros((feature_dim, current_ba"], Modalities selected for training
    "blank_percentage":[0.0], # percentage of spectra that are blanked out during training for better generalizability of the network to various datatypes
    "batch_size":[64], # number needs to be the same as number of GPUs 
    "num_epochs": [10], # number of epochs for training
    "lr_pretraining": [1e-4], # Pretraining learning rate
    "lr_finetuning": [5e-5], # Finetuning learning rate
    "load_model": [True], # if model should be loaded from path
    "checkpoint_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/V8_MMT_MW_Drop/MultimodalTransformer_time_1704760608.6927748_Loss_0.137.ckpt"], #V8


    "save_model": [True], # if model should be saved
    # Model architecture
    "in_size": [len(stoi)],
    "hidden_size": [128],
    "out_size": [len(stoi)],
    "num_encoder_layers": [6], #8
    "num_decoder_layers": [6], #8
    "num_heads": [16], #8  ### number of attention heads
    "forward_expansion": [4], #4
    "max_len": [128], # maximum length of the generated sequence
    "drop_out": [0.1],
    "fingerprint_size": [512], # Dimensions of encoder output for CLIP contrastive training    
    #"track_metrics":[True],
    "gen_SMI_sequence":[True], # If the model generates a sequence with the SMILES current model for evaluation
    "sampling_method":["mix"], # weight_mol_weight ["multinomial", "greedy". "mix"]  
    "training_setup":["pretraining"], # ["pretraining","finetuning"]
    "smi_randomizer":[False], # if smiles are randomized or canonical during training
    ### SGNN Feedback
    "sgnn_feedback":[False], # if SGNN generates 1H and 13C spectrum on the fly on the generated smiles -> "gen_SMI_sequence":[True]
    "matching":["HungDist"], #["MinSum","EucDist","HungDist"], # HSQC point matching technique used
    "padding":["NN"], # ["Zero","Trunc","NN"], # HSQC padding technique used -> see publication: XXX
    # Weight feedback
    "train_weight_min":[None], # Calculate on the fly - Used for the weight loss calculation for scaling
    "train_weight_max":[None], # Calculate on the fly - Used for the weight loss calculation for scaling
    # Training Loss Weighting options
    #"symbol_reward_weight": [0.1], # loss weight if considered to contribute to loss function
    "weight_validity": [0.0], # up to 1
    "weight_SMI": [1.0], # up to 1
    #"weight_MF": [1.0], # up to 1
    "weight_FP": [0.0], # up to 1
    "weight_MW": [0], # up to 100
    "weight_sgnn": [0.0], # up to 10
    "weight_tanimoto": [0.0], # up to 1
    "change_loss_weights":[False], # if selected the weights get ajusted along the training
    "increment":[0.01], # increment on how much it gets ajusted during training -> TODO
    "batch_frequency":[10000], # Frequency how often it gets ajusted -> TODO
    
    ### For Validation
    "beam_size": [1],  
    "multinom_runs": [1], 
    "temperature":[1],
    "gen_len":[64],
    "pkl_save_folder":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/___FIGURES_PAPERS/pkl_save_folder"],
    
    ### Molformer options 
    "MF_max_trails":[500],
    "MF_tanimoto_filter":[0.1],
    "MF_filter_higher":[1], # False = 0 True = 1
    "MF_delta_weight":[5],
    "MF_generations":[30],
    "MF_model_path":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/deep-molecular-optimization/experiments/trained/Alessandro_big/weights_pubchem_with_counts_and_rank_sanitized.ckpt"],
    "MF_vocab":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/deep-molecular-optimization/experiments/trained/Alessandro_big/vocab_new.pkl"],
    "MF_csv_source_folder_location":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/deep-molecular-optimization/data/MMP"],
    "MF_csv_source_file_name":["test_selection_2"],
    "MF_methods":["MMP"], #["MMP", "scaffold", "MMP_scaffold"],    
    "max_scaffold_generations":[10], #
    ### MMT batch generation
    "MMT_batch":[32], # how big is the batch of copies of the same inputs that is processed by MMT 
    "MMT_generations":[4], # need to be multiple of MMT_batch -> number of valid generated molecules
    #------------------------
    "n_samples":[10], # number of molecules that should be processed for data generation - needs to be smaller than dataloader size
    "gen_mol_csv_folder_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/24_SGNN_gen_folder_2"], # number of molecules that should be processed for data generation - needs to be smaller than dataloader size
    
    ### Fine-tuning improvement options
    "train_data_blend":[0], # how many additional molecules should be added to the new dataset from the original training dataset
    "train_data_blend_CLIP":[1000], # how many additional molecules should be added to the new dataset from the original training dataset
    
    ### Data generation SGNN -> 1H, 13C, HSQC, COSY
    "SGNN_gen_folder_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/24_SGNN_gen_folder_2/dump_2"],
    "SGNN_csv_gen_smi":["/projects/cc/knlr326/1_NMR_project/2_Notebooks/MultiModalSpectralTransformer/deep-molecular-optimization/data/MMP/test_selection_1.csv"],
    "SGNN_size_filter":[550],
    "SGNN_csv_save_folder":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/24_SGNN_gen_folder_2"],
    "IR_save_folder":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/24_SGNN_gen_folder_2/IR_data"],
    
    #################################################
    #### LEGACY parameters for other expeirments ####
    #################################################
    #### CLIP Settings ####
    ### ChemBerta
    "model_version":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v1/Chemberta_source"],   # Source of pretrained chemberta from paper
    "CB_model_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v1/Large_300_15.pth"], # path to pretrained Chemberta model
    "num_class":[1024], #
    "num_linear_layers":[0], # number of linear layers in architecture before num_class output
    "use_dropout":[True],
    "use_relu":[False],
    "loss_fn":["BCEWithLogitsLoss"], #"MSELoss", "BCELoss", 
    "CB_embedding": [1024], #1024
    # PCA
    "fp_dim_reduction":[False], #True
    "pca_components":[300],  
    #"CB_model_name": ["Large_300_15"],

    ### Multimodal Transformer
    "MT_model_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/V8_MMT_MW2_Drop/MultimodalTransformer_time_1706856620.3718672_Loss_0.202.pth"],  # path to pretrained Multimodal Transformer model  
    #"MT_model_name": ["SpectrumBERT_PCA_large_3.6"],
    "MT_embedding": [512], #512
    ### Projection Head
    "projection_dim": [512],
    "dropout": [0.1],
    
    #CLIP
    # Dataloader settings
    "similarity_threshold":[0.6], # Filtere that selects just molecules with a tanimotosimilarity higher than that number
    "max_search_size":[10000], # Size of the data that will be searched to find the similar molecules  # 100000
    "weight_delta":[50], # Filter to molecules with a +/- delta weight of that numbeTraceback (most recent call last):
    "CLIP_batch_size":[128],  #,64,128,256 ### batch size for the CLIP training
    "CLIP_NUM_EPOCHS": [10],    # Number of training epochs
    
    ### Train parameters
    ### CLIP Model   
    "CLIP_temperature": [1],
    #"CB_projection_lr": [1e-3], # projection head learning rate for Chemberta
    "MT_projection_lr": [1e-3], # projection head learning rate for Multimodal Transfomer
    "CB_lr": [1e-4], # Chemberta Learning Rate
    "MT_lr": [1e-5], # Multimodal Transfomer Learning Rate
    "weight_decay": [1e-3], # Weight decay for projection heads -> TODO why just on those
    "patience": [1],   # not integrated yet
    "factor": [0.8],   # not integrated yet
    "CLIP_continue_training":[True],
    "CLIP_model_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/V8_modalities_CLIP_1_dot_product/MultimodalCLIP_Epoch_9_Loss0.096.ckpt"],   
    "CLIP_model_save_dir":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/test_CLIP"],
    
    ### BLIP Model
    "BLIP_temperature": [1],
    "Qformer_lr":[1e-4],
    "Qformer_CB_lr":[1e-4],
    "Qformer_MT_lr":[1e-4],
    "BLIP_continue_training":[True],
    "BLIP_model_path":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/test_BLIP_1M/model_BLIP-epoch=03-loss=2.54_v0.ckpt"],   
    "BLIP_model_save_dir":["/projects/cc/knlr326/1_NMR_project/1_NMR_data_AZ/1_old_models/models_v2/test_BLIP_1M"],
    
    }



import pandas as pd
import torch
from torch.utils.data import DataLoader
from argparse import Namespace

def parse_arguments(hyperparameters):
    # Using dictionary comprehension to simplify your code
    parsed_args = {key: val[0] for key, val in hyperparameters.items()}
    return Namespace(**parsed_args)

config = parse_arguments(hyperparameters)

### SGNN file generation

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from tqdm import tqdm

def contains_hydrogen_rdkit(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    if mol:
        for atom in mol.GetAtoms():
            if atom.GetSymbol() == 'H':
                return True
    return False

# Calculate molecular weight
def calculate_mw(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Descriptors.MolWt(mol) if mol is not None else None
    except:
        return None

tqdm.pandas()

# Function to check if a SMILES has disconnected structures
def has_disconnected_smiles(smiles_string):
    return '.' in smiles_string

In [5]:
from utils.sgnn_code_pl_v13 import *
import pandas as pd
from tqdm import tqdm
import glob
from rdkit.Chem import MolFromSmiles
import time


def run_sgnn(config):
    graph_representation = "sparsified"
    target = "13C"
    train_y_mean_C, train_y_std_C = load_std_mean(target,graph_representation)
    target = "1H"
    train_y_mean_H, train_y_std_H = load_std_mean(target,graph_representation)
    sgnn_means_stds = (train_y_mean_C, train_y_std_C, train_y_mean_H, train_y_std_H)


    batch_size = 128
    path_csv = config.SGNN_csv_gen_smi
    ML_save_folder = config.SGNN_gen_folder_path
    data_df = pd.read_csv(path_csv)
    #data_df = data_df.iloc[38449:]
    ### Remove every molecule that does not have any hydrogens
    data_df = data_df[data_df['SMILES'].apply(contains_hydrogen_rdkit)]
    # Filter out disconnected structures
    data_df = data_df[~data_df['SMILES'].apply(has_disconnected_smiles)]  # Note the use of ~ for negation

    data_df['Molecular_Weight'] = data_df['SMILES'].apply(calculate_mw)
    data_df_final = data_df[data_df['Molecular_Weight'] <= config.SGNN_size_filter]

    #data_df['SMILES'] = data_df['smiles']
    #data_df['sample-id'] = data_df['zinc_id']
    #data_df = data_df.iloc[:]#


    if not os.path.exists(ML_save_folder):
        os.mkdir(ML_save_folder)

    batch_data_1, failed_ids_1 = main_execute(data_df_final, sgnn_means_stds, ML_save_folder, batch_size)

    print(len(failed_ids_1))

    # second round just to let out the failed molecules
    batch_size = 1
    data_df_final = data_df_final[data_df_final['sample-id'].isin(failed_ids_1)]
    batch_data_2, failed_ids_2 = main_execute(data_df_final, sgnn_means_stds, ML_save_folder, batch_size)
    batch_data_1, batch_data_2
    combined_df = pd.concat([batch_data_1, batch_data_2], axis=0, ignore_index=True)
    return combined_df

Using backend: pytorch


In [ ]:
#combined_df = run_sgnn(config)

/projects/cc/knlr326/1_NMR_project/2_Notebooks/nmr_project/1_Dataexploration/2_paper_code/Experiments_SLURM/20.0_SLURM_MasterTransformer/utils/sgnn_code_pl_v13.py:697: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if target == '1H': mol_dict['shift'] = np.array(mol_dict['shift'])#, dtype=object)


0


In [ ]:
#combined_df

,sample-id,sdf_path,SMILES
0,sn1151434342,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(NC2CC3CNCC2C3)nc2c1c(=O)n(Cc1nc(C)c3c...
1,sn1151434224,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CCNCC2C)nc2c1c(=O)n(Cc1nccc(C)n1)c(...
2,sn1151434191,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CCC[C@@H](N)C2)nc2c1c(=O)n(Cc1nc(C)...
3,sn1151433837,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CC(N)C2)nc2c1c(=O)n(Cc1nc(C)c3ccccc...
4,sn1151433568,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CC[C@@H]3CNC[C@@H]32)nc2c1c(=O)n(Cc...
5,sn1151432766,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CCC3CCC(C2)N3)nc2c1c(=O)n(Cc1nc(C)c...
6,sn1151432648,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CC(N)C2)nc2c1c(=O)n(Cc1nccc(C)n1)c(...
7,sn1151432516,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CCC[C@H]2CN)nc2c1c(=O)n(Cc1nccc(C)n...
8,sn1151432502,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N2CCC[C@@H]2CN)nc2c1c(=O)n(Cc1nccc(C)...
9,sn1151432483,/projects/cc/knlr326/1_NMR_project/1_NMR_data_...,CC#CCn1c(N[C@H]2C[C@@H](NC)C2)nc2c1c(=O)n(Cc1n...


### 1H Simulation - create CSV file

### 1H functions

In [28]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
import matplotlib.pyplot as plt
from rdkit.Chem import SDMolSupplier, Draw, MolFromSmiles, MolToSmiles
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

# Add this function to read shifts from the SDF file
def read_shifts_from_sdf(file_path):
    supplier = SDMolSupplier(file_path)
    sdf_mol = supplier[0]  # assuming there is only one molecule in the file
    shifts = {}
    for atom in sdf_mol.GetAtoms():
        atom_idx = atom.GetIdx()
        atom_shift = atom.GetProp("_Shift")
        shifts[atom_idx] = float(atom_shift)
    return shifts

def lorentzian(x, x0, gamma):
    return (1 / np.pi) * (0.5 * gamma) / ((x - x0) ** 2 + (0.5 * gamma) ** 2)


def simulate_splitting(shifts, coupling_patterns, gamma, spectrometer_frequency):
    x = np.linspace(shifts.min() - 1, shifts.max() + 1, 1000)
    y = np.zeros_like(x)
    for shift, coupling_pattern in zip(shifts, coupling_patterns):
        peak = np.zeros_like(x)
        for J, intensity in coupling_pattern:
            peak += intensity * lorentzian(x, shift + J / spectrometer_frequency, gamma)
        y += peak
    return x, y

def get_adjacent_aromatic_hydrogens(atom):
    aromatic_neighbors = [neighbor for neighbor in atom.GetNeighbors() if neighbor.GetIsAromatic()]
    aromatic_hydrogens = []
    for aromatic_neighbor in aromatic_neighbors:
        aromatic_hydrogens.extend(get_surrounding_hydrogens(aromatic_neighbor))
    return aromatic_hydrogens

def get_surrounding_hydrogens(atom):
    neighboring_hydrogens = []
    for neighbor in atom.GetNeighbors():
        if neighbor.GetSymbol() == 'H':
            neighboring_hydrogens.append((neighbor, neighbor.GetIdx()))
    return neighboring_hydrogens

from rdkit.Chem import Draw
# Updated analyze_molecule function to combine symmetric hydrogen shifts
def analyze_molecule(mol):
    hydrogens = [atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'H']
    nmr_data = []
    assigned_shifts = {}

    for hydrogen in hydrogens:
        parent_atom = hydrogen.GetNeighbors()[0]
        is_aromatic = parent_atom.GetIsAromatic()
        group_key = (parent_atom.GetSymbol(), parent_atom.GetIdx())

        surrounding_hydrogens = get_surrounding_hydrogens(parent_atom)
        num_h_neighbors = len(surrounding_hydrogens) - 1

        hydrogen_label = f"{parent_atom.GetSymbol()}{parent_atom.GetIdx()}H{num_h_neighbors + 1 - surrounding_hydrogens.count(hydrogen)}"

        nmr_data.append({
            'atom': hydrogen,
            'aromatic': is_aromatic,
            'neighbors': num_h_neighbors,
            'label': hydrogen_label,
            'group_key': group_key
        })

        assigned_shifts[group_key] = float(hydrogen.GetProp("_Shift"))
    
    return nmr_data, assigned_shifts, mol


def pascals_triangle(n):
    if n == 0:
        return [1]
    else:
        previous_row = pascals_triangle(n - 1)
        current_row = [1]
        for i in range(len(previous_row) - 1):
            current_row.append(previous_row[i] + previous_row[i + 1])
        current_row.append(1)
        return current_row

def generate_nmr_coupling_pattern(n_neighbors, J):
    coefficients = pascals_triangle(n_neighbors)
    intensities = [coef / (2 ** n_neighbors) for coef in coefficients]
    Js = [i * J for i in range(-n_neighbors // 2, n_neighbors // 2 + 1)]
    return list(zip(Js, intensities))

import pandas as pd
from rdkit.Chem import PandasTools
from rdkit.Chem import AddHs


def load_mol_and_assign_shifts(file_path):
    data = PandasTools.LoadSDF(file_path)
    mol = data["ROMol"].item()
    mol = AddHs(mol, addCoords=True)

    str_shifts = data["averaged_NMR_shifts"].item()
    shifts  = [float(i) for i in str_shifts.split()]

    atoms = list(mol.GetAtoms())
    i = 0
    for idx, atom in enumerate(atoms):
        atom.SetProp("_Shift", str(shifts[idx]))
    mol = AddHs(mol, addCoords=False)

    return mol



def add_shifts_to_data(nmr_data, assigned_shifts):
    ### Calculate the average shift to the nmr_data dictionary 
    ### where multiple H are attached to one Carbon
    grouped_shifts = {}
    for atom_data in nmr_data:
        group_key = atom_data['group_key']
        if group_key not in grouped_shifts:
            grouped_shifts[group_key] = []
        if group_key in assigned_shifts:  
            # Check if the key is in assigned_shifts before appending
            grouped_shifts[group_key].append(assigned_shifts[group_key])
    
    # Take the average of shifts in each group
    avg_shifts = {group_key: np.mean([shift for shift in shifts if np.isfinite(shift)]) for group_key, shifts in grouped_shifts.items()}

    # Replace the original shifts with the average shifts
    for atom_data in nmr_data:
        group_key = atom_data['group_key']
        if group_key in avg_shifts:  
            # Check if the key is in avg_shifts before assigning
            atom_data['shift'] = avg_shifts[group_key]
            ### This creates duplicates entries for the hydrogens that are on the same Carbon
    return nmr_data

def calculate_couplings_constants(nmr_data):
    
    J_aromatic = 8.0

    ### Version 2
    # Calculate coupling patterns using the average shifts
    atoms_done = []
    coupling_patterns = []
    hydrogen_num = []
    shifts = []
    hydrogen_counts = None
    
    for atom_data in nmr_data:
        if ("N" in atom_data["label"] or "O" in atom_data["label"]) :
            continue
        parent_atom = atom_data['atom'].GetNeighbors()[0]
        if (atom_data['aromatic'] and atom_data['label'] not in atoms_done):
            n_neighbors = atom_data['neighbors']
            adjacent_aromatic_hydrogens = get_adjacent_aromatic_hydrogens(parent_atom)
            arom_n_neighbors = len(adjacent_aromatic_hydrogens)
            if arom_n_neighbors == 0:
                coupling_patterns.append([(J_aromatic, 1)])
            else:
                coupling_patterns.append(generate_nmr_coupling_pattern(arom_n_neighbors, J_aromatic))
            shifts.append(atom_data['shift'])
            atoms_done.append(atom_data['label'])
            hydrogen_num.append(atom_data['neighbors']+1)
        elif atom_data['label'] not in atoms_done:

            bond_types = [bond.GetBondType() for bond in parent_atom.GetBonds() if bond.GetOtherAtom(parent_atom).GetSymbol() == 'C']        

            n_neighbors = atom_data['neighbors']

            carbon_neighbors = [neighbor for neighbor in parent_atom.GetNeighbors() if neighbor.GetSymbol() == 'C']

            hydrogen_counts = [sum(1 for neighbor in carbon_neighbor.GetNeighbors() if neighbor.GetSymbol() == 'H') for carbon_neighbor in carbon_neighbors]
            #print(hydrogen_counts, atom_data["label"])
            # Rule-based coupling pattern generation
            if hydrogen_counts == [] and n_neighbors == 2:
                #N-CH3
                coupling_pattern = [(0, 3)]   

            if hydrogen_counts == [] and n_neighbors == 1:
                #CCl2=CH2
                coupling_pattern = [(0, 2)]  
                
            if hydrogen_counts == [] and n_neighbors == 0:
                #(CCl2)3-CH
                coupling_pattern = [(0, 0)]  

            if hydrogen_counts == [0] and n_neighbors == 2:
                coupling_pattern = [(0, 3)]

            if hydrogen_counts == [0] and n_neighbors == 1:
                coupling_pattern = [(0, 2)]   

            if hydrogen_counts == [0] and n_neighbors == 0:
                coupling_pattern = [(0, 1)]  


            if hydrogen_counts == [0, 0] and n_neighbors == 0:
                coupling_pattern = [(0, 1)]  
            
            if hydrogen_counts == [0, 0] and n_neighbors == 1:
                coupling_pattern = [(0, 2)]   

            if hydrogen_counts == [1] and Chem.rdchem.BondType.DOUBLE in bond_types and n_neighbors == 1:
                # CH=CH2 case J = 16 10
                ### Approximation
                J_doublet_1 = 16  
                J_doublet_2 = 10 
                coupling_pattern = [(-0.5*J_doublet_1-0.5*J_doublet_2, 1/2), 
                                    (-0.5*J_doublet_1+0.5*J_doublet_2, 1/2),
                                    (0.5*J_doublet_1+0.5*J_doublet_2, 1/2), 
                                    (0.5*J_doublet_1-0.5*J_doublet_2, 1/2)]

            elif hydrogen_counts == [1] and Chem.rdchem.BondType.SINGLE in bond_types and n_neighbors == 1:
                # CH-CH2-Cl case J=5.9
                J_doublet = 5.9  # Coupling constant for the single bond between the CH hydrogens
                coupling_pattern = [(-0.5*J_doublet, 1), 
                                      (0.5*J_doublet, 1)]

            elif hydrogen_counts == [1] and Chem.rdchem.BondType.SINGLE in bond_types and n_neighbors == 2:
                # CH-CH3 case J = 6.1
                J_doublet = 6.1  # Coupling constant for the single bond between the CH hydrogens
                coupling_pattern = [(-0.5*J_doublet, 1.5), 
                                      (0.5*J_doublet, 1.5)]
                
            if hydrogen_counts == [1] and n_neighbors == 0:
                #(CCl2)2-CH-CHCl2                
                J_doublet = 6.1
                coupling_pattern = [(-0.5*J_doublet, 0.5),
                                    (0.5*J_doublet, 0.5)]  
                
            #elif hydrogen_counts == [2] and Chem.rdchem.BondType.DOUBLE in bond_types:
            #    # CH2=CH2 case
            #    J_triplet = J_double_bond  # Coupling constant for the double bond between the CH2=CH2 hydrogens
            #    coupling_pattern = [(-J_triplet, 1/2), 
            #                          (0, 2/2), 
            #                          (J_triplet, 1/2)]

            elif hydrogen_counts == [2] and Chem.rdchem.BondType.SINGLE in bond_types:
                # CH2-CH2 case J = 6.3
                J_triplet = 6.3  # Coupling constant for the single bond between the CH2-CH2 hydrogens
                coupling_pattern = [(-J_triplet, 1/2), 
                                      (0, 2/2), 
                                      (J_triplet, 1/2)]

            elif hydrogen_counts == [3] and Chem.rdchem.BondType.SINGLE in bond_types:
                # CH3-CH2 case J = 7
                J_quartet =   7.0
                coupling_pattern = [(-1.5*J_quartet, 2/6), 
                                      (-0.5*J_quartet, 4/6), 
                                      (0.5*J_quartet, 4/6), 
                                      (1.5*J_quartet, 2/6)]

            elif (hydrogen_counts == [1,0] or hydrogen_counts == [0,1]):
                # CH-CH2-CO case J = 7
                J_doublet = 6.9  
                coupling_pattern = [(-0.5*J_doublet, 1), 
                                      (0.5*J_doublet, 1)]            


            elif (hydrogen_counts == [2,0] or hydrogen_counts == [0,2]) and Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH2=CH-CO case J = 7
                J_doublet_1 = 18  # Coupling constant for the double bond between the CH=CH2 hydrogens
                J_doublet_2 = 10  # Coupling constant for the double bond between the CH=CH2 hydrogens
                coupling_pattern = [(-0.5*J_doublet_1-0.5*J_doublet_2, 1/4), 
                                    (-0.5*J_doublet_1+0.5*J_doublet_2, 1/4),
                                    (0.5*J_doublet_1-0.5*J_doublet_2, 1/4), 
                                    (0.5*J_doublet_1+0.5*J_doublet_2, 1/4)]  

            elif (hydrogen_counts == [2,0] or hydrogen_counts == [0,2]) and not Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH2-CH2-CO case J = 7
                J_triplet = 6.7  # Coupling constant for the double bond between the CH=CH2 hydrogens
                coupling_pattern = [(-J_triplet, 1/2), 
                                      (0, 2/2), 
                                      (J_triplet, 1/2)]   

            elif hydrogen_counts == [3,0]  or hydrogen_counts == [0,3]:
                # CH3-CHCl-CO case J = 7
                J_quartet = 7  
                coupling_pattern = [(-1.5*J_quartet, 1/6), 
                                      (-0.5*J_quartet, 2/6), 
                                      (0.5*J_quartet, 2/6), 
                                      (1.5*J_quartet, 1/6)]
                
            elif hydrogen_counts == [1, 1] and Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH-CH=CH non-aromatic case  13/6.06
                J_doublet_1 = 6.06  # Coupling constant for the single bond between CH hydrogens
                J_doublet_2 = 13 # Coupling constant for the double bond between the CH=CH hydrogens
                coupling_pattern = [(-0.5*J_doublet_1-0.5*J_doublet_2, 1/4), 
                                    (-0.5*J_doublet_1+0.5*J_doublet_2, 1/4),
                                    (0.5*J_doublet_1-0.5*J_doublet_2, 1/4), 
                                    (0.5*J_doublet_1+0.5*J_doublet_2, 1/4)]

            elif hydrogen_counts == [1, 1] and not Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH-CH2-CH non-aromatic case  13/6.06
                J_triplet = 6.0  # Coupling constant for the double bond between the CH=CH2 hydrogens
                coupling_pattern = [(-J_triplet, 1/2), 
                                      (0, 2/2), 
                                      (J_triplet, 1/2)]   

            elif (hydrogen_counts == [1, 2] or hydrogen_counts == [2, 1]) and bond_types.count(Chem.rdchem.BondType.SINGLE) == 1 and bond_types.count(Chem.rdchem.BondType.DOUBLE) == 1:  
                # CH=CH-CH2 case  J = 7.4 
                J_quartet = 7.4  # Coupling constant for the single bond between the CH3-CH3 hydrogens
                coupling_pattern = [(-1.5*J_quartet, 1/6), 
                                      (-0.5*J_quartet, 2/6), 
                                      (0.5*J_quartet, 2/6), 
                                      (1.5*J_quartet, 1/6)]

            elif (hydrogen_counts == [1, 2] or hydrogen_counts == [2, 1]) and bond_types.count(Chem.rdchem.BondType.SINGLE) == 2:  
                # CH-CH2-CH2 case  J = 7.4 
                # an approximation
                J_quartet = 7.4   # Coupling constant for the single bond between the CH3-CH3 hydrogens
                coupling_pattern = [(-1.5*J_quartet, 2/6), 
                                      (-0.5*J_quartet, 4/6), 
                                      (0.5*J_quartet, 4/6), 
                                      (1.5*J_quartet, 2/6)]

            elif hydrogen_counts == [2, 2] and bond_types.count(Chem.rdchem.BondType.SINGLE) == 2:  
                # CH2-CH2-CH2 case (quintet)  J=6.57
                J_quintet = 6.57
                coupling_pattern = [(-2 * J_quintet, 2/9), 
                                    (-J_quintet, 4/9), 
                                    (0, 6/9), 
                                    (J_quintet, 4/9), 
                                    (2 * J_quintet, 2/9)]

            elif (hydrogen_counts == [3, 1] or hydrogen_counts == [1, 3]) and Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH3-CH=CH case or CH=CH-CH3 J = 7 (Douplet of quartet)
                # an approximation
                J_quintet = 7
                coupling_pattern = [(-2 * J_quintet, 1/9), 
                                    (-J_quintet, 2/9), 
                                    (0, 3/9), 
                                    (J_quintet, 2/9), 
                                    (2 * J_quintet, 1/9)]

            elif (hydrogen_counts == [3, 1] or hydrogen_counts == [1, 3]) and not Chem.rdchem.BondType.DOUBLE in bond_types:
                # CH3-CH-CHCl case or CH=CH-CH3 J = 7 (Douplet of quartet)
                # an approximation
                J_octet = 3.5
                coupling_pattern = [(-3.5*J_octet, 1/12), 
                                   (-2.5*J_octet, 1/12),  
                                  (-1.5*J_octet, 2/12), 
                                  (-0.5*J_octet, 2/12), 
                                  (0.5*J_octet, 2/12), 
                                  (1.5*J_octet, 2/12), 
                                  (2.5*J_octet, 1/12),
                                  (3.5*J_octet, 1/12)]


            elif (hydrogen_counts == [3, 2] or hydrogen_counts == [2, 3]) and bond_types.count(Chem.rdchem.BondType.SINGLE) == 2:  
                # CH3-CH2-CH2 case (Quartet of Triplets) most likely like a sextet
                # an approximation
                J_sixtet = 7
                coupling_pattern = [(-2.5*J_sixtet, 2/12), 
                                      (-1.5*J_sixtet, 4/12), 
                                      (-0.5*J_sixtet, 6/12), 
                                      (0.5*J_sixtet, 6/12),
                                      (1.5*J_sixtet, 4/12), 
                                      (2.5*J_sixtet, 2/12)]

            elif hydrogen_counts == [3, 3]:
                # CH3-CH-CH3 with another connection to CH with CH3 J= 6.4
                J_septet = 6.4  # Coupling constant between the central CH hydrogen and the CH3 hydrogens
                coupling_pattern = [(-3*J_septet, 1/16), 
                                   (-2*J_septet, 2/16),  
                                  (-1*J_septet, 3/16), 
                                  (0*J_septet, 4/16), 
                                  (1*J_septet, 3/16), 
                                  (2*J_septet, 2/16),
                                  (3*J_septet, 1/16)]
            elif hydrogen_counts == [0, 0, 0]:
                coupling_pattern = [(0, 1)]  

            elif hydrogen_counts == [0, 0, 1] or hydrogen_counts == [0, 1, 0] or hydrogen_counts == [1, 0, 0]:
                # (CCl3)2-CH-CHCl2
                J_doublet = 6.1 
                coupling_pattern = [(-0.5*J_doublet, 0.5), 
                                  (0.5*J_doublet, 0.5)]  
                
            elif (hydrogen_counts == [0, 1, 1] or hydrogen_counts == [1,1,0] or hydrogen_counts == [1,0,1]):
                #COCl-CH-(CHCl)2
                J_triplet = 7  # Coupling constant for the double bond between the CH=CH2 hydrogens
                coupling_pattern = [(-J_triplet, 1/4), 
                                      (0, 2/4), 
                                      (J_triplet, 1/4)]   
                
            elif (hydrogen_counts == [0,2,2] or hydrogen_counts == [2,2,0] or hydrogen_counts == [2,0,2]):
                #COCl-CH-(CH2)
                J_quintet = 7.5
                coupling_pattern = [(-2 * J_quintet, 1/9), 
                                    (-J_quintet, 2/9), 
                                    (0, 3/9), 
                                    (J_quintet, 2/9), 
                                    (2 * J_quintet, 1/9)]
            elif (hydrogen_counts ==  [0, 2, 0] 
                   or hydrogen_counts ==  [0, 0, 2] 
                   or hydrogen_counts ==  [2, 0, 0]):
                # Approximation dd ->t
                J_triplet = 7  # Coupling constant for the double bond between the CH=CH2 hydrogens
                coupling_pattern = [(-J_triplet, 1/4), 
                                      (0, 2/4), 
                                      (J_triplet, 1/4)]   
            
            elif (hydrogen_counts ==  [0, 2, 1] 
                   or hydrogen_counts ==  [0, 1, 2] 
                   or hydrogen_counts ==  [1, 2, 0]
                   or hydrogen_counts ==  [1, 0, 2]
                   or hydrogen_counts ==  [2, 0, 1]
                   or hydrogen_counts ==  [2, 1, 0]):
                #COCl-CH-(CH2)(CH) ddd
                # an approximation
                J_quartet = 7.0   # Coupling constant for the single bond between the CH3-CH3 hydrogens
                coupling_pattern = [(-1.5*J_quartet, 1/6), 
                                      (-0.5*J_quartet, 2/6), 
                                      (0.5*J_quartet, 2/6), 
                                      (1.5*J_quartet, 1/6)]

            elif (hydrogen_counts == [1,2,2]
                   or hydrogen_counts ==  [2, 1, 2] 
                   or hydrogen_counts ==  [2, 2, 1]):
                    #CH-CH-(CH2)2  ttd
                J_septet = 6.2  # Coupling constant between the central CH hydrogen and the CH3 hydrogens
                J_12 = 3
                coupling_pattern = [(-5.5*J_12, 1/42), 
                                   (-4.5*J_12, 2/42),  
                                  (-3.5*J_12, 3/42), 
                                  (-2.5*J_12, 4/42), 
                                  (-1.5*J_12, 5/42), 
                                  (-0.5*J_12, 6/42),
                                  (0.5*J_12, 6/42), 
                                  (1.5*J_12, 5/42), 
                                  (2.5*J_12, 4/42),
                                  (3.5*J_12, 3/42), 
                                  (4.5*J_12, 2/42), 
                                  (5.5*J_12, 1/42)]   
                
            elif hydrogen_counts == [2,2,2]:
                #CH2-CH-(CH2)2
                J_septet = 6.2  # Coupling constant between the central CH hydrogen and the CH3 hydrogens
                coupling_pattern = [(-3*J_septet, 1/16), 
                                   (-2*J_septet, 2/16),  
                                  (-1*J_septet, 3/16), 
                                  (0*J_septet, 4/16), 
                                  (1*J_septet, 3/16), 
                                  (2*J_septet, 2/16),
                                  (3*J_septet, 1/16)]            

        try:
            if hydrogen_counts != None:
                coupling_patterns.append(coupling_pattern)
                atoms_done.append(atom_data['label'])
                shifts.append(atom_data['shift'])
                hydrogen_num.append(atom_data['neighbors']+1)
            else:
                continue
        except:
            print(hydrogen_counts, n_neighbors)

        #if atom_data["label"] =="C1H1":
        #    break
    return coupling_patterns, atoms_done, shifts, hydrogen_num


def create_plot_NMR(shifts, coupling_patterns, gamma, spectrometer_frequency):
    x, y = simulate_splitting(np.array(shifts), coupling_patterns, gamma, spectrometer_frequency)
    plt.plot(x, y)
    plt.xlabel('Chemical shift (ppm)')
    plt.ylabel('Intensity')
    for shift, label in zip(shifts, atoms_done):
        plt.text(shift, np.max(lorentzian(x, shift, gamma)), label, ha='center', va='bottom', fontsize=8, rotation=45)

    plt.gca().invert_xaxis()
    plt.show()
    
def create_plot_NMR_interactiv(shift_intensity_label_data):
    
    # Create an interactive plot using Plotly
    fig = go.Figure()

    for shift, intensity, label in shift_intensity_label_data:
        fig.add_trace(
            go.Scatter(
                x=[shift, shift],  # Use two points to create a vertical line
                y=[0, intensity],
                mode="lines",
                line=dict(color="black", width=1.5),
                hoverinfo="none",  # Disable hover info
            )
        )

    # Find the maximum intensity for each shift
    max_intensities_dict = {}
    for item in shift_intensity_label_data:
        _, intensity, label = item
        if label not in max_intensities_dict:
            max_intensities_dict[label] = intensity
        else:
            max_intensities_dict[label] = max(max_intensities_dict[label], intensity)
    max_intensities = list(max_intensities_dict.values())


    # Add a separate trace for the labels
    fig.add_trace(
        go.Scatter(
            x=shifts,
            y=max_intensities,
            mode="text",
            text=atoms_done,
            textposition="top center",
            hoverinfo="none",  # Disable hover info
        )
    )

    fig.update_layout(
        xaxis=dict(title="Chemical shift (ppm)", range=[11, 0]),  # Set x-axis range
        yaxis=dict(title="Intensity"),
        showlegend=False,  # Remove the legend from the plot
    )

    multiplicity_labels = []
    for coupling_pattern in coupling_patterns:
        if len(coupling_pattern) == 1:
            multiplicity_labels.append("Singlet")
        else:
            n_split = len(coupling_pattern)
            if n_split == 2:
                multiplicity_labels.append("Doublet")
            elif n_split == 3:
                multiplicity_labels.append("Triplet")
            elif n_split == 4:
                multiplicity_labels.append("Quartet")
            elif n_split == 5:
                multiplicity_labels.append("Quintet")
            elif n_split>5:
                multiplicity_labels.append(f"{n_split} peaks")

    fig.add_trace(
        go.Scatter(
            x=shifts,
            y=[intensity - 0.1 * intensity for intensity in max_intensities],
            mode="text",
            text=multiplicity_labels,
            textposition="bottom center",
            textfont=dict(color="red"),  # Use a different color for multiplicity labels
            hoverinfo="none",  # Disable hover info
        )
    )
    fig.show()
    
def create_labeled_structure(mol,assigned_shifts):

    import plotly.graph_objs as go
    from rdkit.Chem import rdDepictor

    # Generate a 2D depiction of the molecule to better fit the drawing canvas
    rdDepictor.Compute2DCoords(mol)

    # Create a MolDraw2DSVG object to draw the molecule as an SVG
    # You can adjust the width and height values to better fit the molecule
    drawer = rdMolDraw2D.MolDraw2DSVG(600, 200)
    opts = drawer.drawOptions()

    # Set atom labels based on the assigned_shifts dictionary
    for (atom_type, atom_idx) in assigned_shifts.keys():
        atom = mol.GetAtomWithIdx(atom_idx)
        opts.atomLabels[atom_idx] = f"{atom_type}{atom_idx}"

    # Draw the molecule
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()

    # Display the SVG in the notebook
    svg = SVG(drawer.GetDrawingText())
    return svg                     
                             
def create_shift_intensity_label_data(shifts, coupling_patterns, atoms_done, spectrometer_frequency):
    shift_intensity_label_data = []
    shift_intensity_data = []
    for shift, coupling_pattern, label in zip(shifts, coupling_patterns, atoms_done):
        if shift!=0.0:   ### Because of ACD labs sometimes there are 0.0 that we used for padding to the right number. these should be ignored
            for J, intensity in coupling_pattern:
                if len(coupling_pattern) > 1:
                    shift1 = shift + (J / spectrometer_frequency)
                else:
                    shift1 = shift
                shift_intensity_label_data.append((shift1, intensity, label))
                shift_intensity_data.append((shift1, intensity))
    return shift_intensity_label_data, shift_intensity_data

In [29]:
#import plotly.graph_objs as go
from IPython.display import display
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG, display
import os
from tqdm import tqdm

def run_1H_generation(config):
    folder_path = config.SGNN_gen_folder_path
    nmr_files = glob.glob(folder_path+"/*")
    nmr_files = [file for file in nmr_files if "NMR_" in file.split("/")[-1] ]
    nmr_files = [file for file in nmr_files if not ".mol" in file]
    nmr_files = sorted(nmr_files, reverse=True)


    ### Settings
    spectrometer_frequency = 400  # Example spectrometer frequency in MHz
    gamma = 0.01
    plot_NMR = False
    plot_NMR_interactiv = False
    show_labeled_structure = False

    ### Stores
    smiles_list = []
    data_list = []
    sample_id_list = []

    for file_path in tqdm(nmr_files[:]):
        try:
            mol = load_mol_and_assign_shifts(file_path)
            #sample_id = file_path[-19:-4]

            # Extract the filename from the path
            file_name = os.path.basename(file_path)

            # Remove the file extension to get the desired part
            sample_id = os.path.splitext(file_name)[0].split('NMR_')[-1]

            #sample_id = file_path[-17:-4] ### For zinc dataset
            nmr_data, assigned_shifts, mol = analyze_molecule(mol)
            nmr_data = add_shifts_to_data(nmr_data, assigned_shifts)
            coupling_patterns, atoms_done, shifts, hydrogen_num = calculate_couplings_constants(nmr_data)
            if plot_NMR:
                create_plot_NMR(shifts, coupling_patterns, gamma, spectrometer_frequency)
            shift_intensity_label_data, shift_intensity_data = create_shift_intensity_label_data(shifts, coupling_patterns, atoms_done, spectrometer_frequency)
            if plot_NMR_interactiv:
                create_plot_NMR_interactiv(shift_intensity_label_data)
            if show_labeled_structure:
                svg_output = create_labeled_structure(mol,assigned_shifts)
                display(svg_output)
            if len(shift_intensity_data)!=0:
                mol = Chem.RemoveHs(mol)
                smi = MolToSmiles(mol)            
                smiles_list.append(smi)
                ### Use a set because symmetric H on C will come for every atom the same
                data_shifts_ints = sorted(list(set(shift_intensity_data)), reverse=False)
                data_list.append(data_shifts_ints)
                sample_id_list.append(sample_id)
            else:
                print(smi, sample_id)
        except:
            import IPython; IPython.embed();
            print(file_path)


    # Create a DataFrame with the lists as columns
    data = pd.DataFrame({
        'SMILES': smiles_list,
        'shifts': data_list,
        'sample-id': sample_id_list,
    })

    data.reset_index(drop=True, inplace=True)
    csv_1H_path = os.path.join(config.SGNN_csv_save_folder, "data_1H.csv")
    data.to_csv(csv_1H_path, index=False)
    return data

### 13C functions

In [32]:
def find_symmetric_positions(stereo_smi):
    """https://github.com/rdkit/rdkit/issues/1411"""

    mol = Chem.MolFromSmiles(stereo_smi)
    z=list(rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))
    matches = mol.GetSubstructMatches(mol, uniquify=False)
    
    if len(z) != len(set(z)) and len(matches) > 1:
    # if len(matches) > 1:

        # # Get a list with all the duplicate numbers
        symmetric = [item for item, count in collections.Counter(z).items() if count > 1]
        
        # Get a list of lists with the positions of the duplicates in match list
        all_duplicates = []
        for j in symmetric:
            indices = [i for i, v in enumerate(z) if v == j]
            all_duplicates.append(indices)
        
        # Get a list of lists with the positions of the duplicates
        example_match = matches[0]
        sym_dupl_lists = []
        for sub_list in all_duplicates:
            indices_list = []
            for i in sub_list:
                position = example_match[i]
                indices_list.append(position)
            sym_dupl_lists.append(indices_list)
    
    else:
        sym_dupl_lists = []
    return sym_dupl_lists


# Function to average the symmetric peaks and update the list
def consolidate_peaks(averaged_shifts, symmetric_positions):
    # Make a copy of the original list to avoid modifying it in place
    consolidated_shifts = averaged_shifts.copy()

    for positions in symmetric_positions:
        # Calculate the average for the symmetric positions
        avg_value = sum(averaged_shifts[i] for i in positions) / len(positions)
        
        # Update the peaks at these positions with the average value
        for i in positions:
            consolidated_shifts[i] = avg_value

    return consolidated_shifts


In [36]:
from IPython.display import display
from rdkit.Chem import MolFromSmiles, MolToSmiles, rdmolfiles
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG, display
import collections
import os
from tqdm import tqdm
from rdkit.Chem.EnumerateStereoisomers import GetStereoisomerCount,EnumerateStereoisomers


def run_13C_generation(config):
    
    folder_path = config.SGNN_gen_folder_path
    nmr_files = glob.glob(folder_path+"/*")
    nmr_files = [file for file in nmr_files if "NMR_" in file.split("/")[-1] ]
    nmr_files = [file for file in nmr_files if not ".mol" in file]
    nmr_files = sorted(nmr_files, reverse=True)

    ### Stores
    smiles_list = []
    data_list = []
    sample_id_list = []


    for file_path in tqdm(nmr_files[:]):
        try:
            #mol = load_mol_and_assign_shifts(file_path)
            mol = SDMolSupplier(file_path)[0]
            isomers = tuple(EnumerateStereoisomers(mol))
            stereo_smi = Chem.MolToSmiles(isomers[0],isomericSmiles=True)

            averaged_nmr_shifts = mol.GetProp('averaged_NMR_shifts')
            sample_shifts = list(map(float, averaged_nmr_shifts.split()))


            # Extract the sample_id from the path
            file_name = os.path.basename(file_path)
            sample_id = os.path.splitext(file_name)[0].split('NMR_')[-1]

            # Remove symmetric C
            sym_dupl_lists = find_symmetric_positions(stereo_smi)
            sym_corr_nmr_shifts = consolidate_peaks(sample_shifts, sym_dupl_lists)

            # Initialize counter for non-hydrogen atoms
            non_hydrogen_count = 0
            # Loop through each atom and count non-hydrogen atoms
            for atom in mol.GetAtoms():
                if atom.GetSymbol() != 'H':
                    non_hydrogen_count += 1

            heavy_atoms_shifts = sym_corr_nmr_shifts[:non_hydrogen_count]
            # Remove zeros
            C_atoms_shifts = [x for x in heavy_atoms_shifts if x != 0]

            # Remove symmetric peaks
            C_atoms_shifts = sorted(list(set(C_atoms_shifts)), reverse=False)

            mol = Chem.RemoveHs(mol)
            smi = MolToSmiles(mol)     

            smiles_list.append(smi)
            data_list.append(C_atoms_shifts)
            sample_id_list.append(sample_id)
        except:
            print(smi, sample_id)


    # Create a DataFrame with the lists as columns
    data = pd.DataFrame({
        'SMILES': smiles_list,
        'shifts': data_list,
        'sample-id': sample_id_list,
    })

    csv_13C_path = os.path.join(config.SGNN_csv_save_folder, "data_13C.csv")
    data.to_csv(csv_13C_path)
    return data

In [37]:
#data = run_13C_generation(config)

100%|██████████| 10/10 [00:00<00:00, 298.98it/s]


### COSY functions

In [39]:
def find_chiral_centers(molecule):
    chiral_centers = []
    
    # Loop through all atoms in the molecule
    for atom in molecule.GetAtoms():
        
        # Get the atomic number and the index (atom number) of the atom
        atomic_num = atom.GetAtomicNum()
        atom_idx = atom.GetIdx()
        
        # Check for chiral tag
        chiral_tag = atom.GetChiralTag()
        
        # Carbon atoms with atomic number 6 are the most common chiral centers
        if atomic_num == 6 and chiral_tag != Chem.ChiralType.CHI_UNSPECIFIED:
            chiral_centers.append(atom_idx)
            
    return chiral_centers

from rdkit import Chem
def find_carbons_with_relevant_neighbors(molecule):
    carbon_dict = {}
    
    for atom in molecule.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        atom_idx = atom.GetIdx()
        
        # Check if the atom is a carbon atom
        if atomic_num == 6:
            neighbor_carbons_with_hydrogens = []
            
            # Loop through the neighbors of the current atom
            for neighbor in atom.GetNeighbors():
                neighbor_idx = neighbor.GetIdx()
                neighbor_atomic_num = neighbor.GetAtomicNum()
                
                # Check if the neighbor is also a carbon atom and both have hydrogens
                if neighbor_atomic_num == 6 and neighbor.GetTotalNumHs() > 0 and atom.GetTotalNumHs() > 0:
                    neighbor_carbons_with_hydrogens.append(neighbor_idx)
            
            # Include carbons with hydrogens, even if they don't couple with any other
            if atom.GetTotalNumHs() > 0:
                carbon_dict[atom_idx] = neighbor_carbons_with_hydrogens
                
    return carbon_dict


from rdkit import Chem
def find_heavy_atoms_with_hydrogens(molecule):
    heavy_atom_dict = {}
    
    # Loop through all atoms in the molecule
    for atom in molecule.GetAtoms():
        atom_idx = atom.GetIdx()
        atomic_num = atom.GetAtomicNum()
        
        # Check if the atom is a heavy atom (i.e., not hydrogen)
        if atomic_num != 1:
            num_hydrogens = atom.GetTotalNumHs()
            
            if num_hydrogens > 0:
                heavy_atom_dict[atom_idx] = num_hydrogens
                
    return heavy_atom_dict


def extract_symmetric_hydrogen_shifts(shifts, heavy_atom_dict):
    num_heavy_atoms = len(heavy_atom_dict)
    
    # Splitting the shifts into heavy atom shifts and hydrogen shifts
    heavy_atom_shifts = shifts[:num_heavy_atoms]
    hydrogen_shifts = shifts[num_heavy_atoms:]
    
    # Initialize the dictionary to store the carbon number and the shift of the attached hydrogens
    carbon_hydrogen_shifts_dict = {}
    
    # Iterate over the heavy_atom_dict in reverse order to pick the hydrogen shifts
    for carbon, num_hydrogens in sorted(heavy_atom_dict.items(), key=lambda x: x[0], reverse=True):
        # Pick the last 'num_hydrogens' from the hydrogen_shifts list
        attached_hydrogen_shifts = hydrogen_shifts[-num_hydrogens:]
        
        # Remove the picked hydrogen shifts from the list
        hydrogen_shifts = hydrogen_shifts[:-num_hydrogens]
        
        # Store in the resulting dictionary
        carbon_hydrogen_shifts_dict[carbon] = attached_hydrogen_shifts
    
    return carbon_hydrogen_shifts_dict


def find_symmetric_positions(stereo_smi):
    """https://github.com/rdkit/rdkit/issues/1411"""

    mol = Chem.MolFromSmiles(stereo_smi)
    z=list(rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))
    matches = mol.GetSubstructMatches(mol, uniquify=False)
    
    if len(z) != len(set(z)) and len(matches) > 1:
    # if len(matches) > 1:

        # # Get a list with all the duplicate numbers
        symmetric = [item for item, count in collections.Counter(z).items() if count > 1]
        
        # Get a list of lists with the positions of the duplicates in match list
        all_duplicates = []
        for j in symmetric:
            indices = [i for i, v in enumerate(z) if v == j]
            all_duplicates.append(indices)
        
        # Get a list of lists with the positions of the duplicates
        example_match = matches[0]
        sym_dupl_lists = []
        for sub_list in all_duplicates:
            indices_list = []
            for i in sub_list:
                position = example_match[i]
                indices_list.append(position)
            sym_dupl_lists.append(indices_list)
    
    else:
        sym_dupl_lists = []
    return sym_dupl_lists

# Function to check if an atom has hydrogens attached to it
def has_hydrogens(mol, atom_idx):
    atom = mol.GetAtomWithIdx(atom_idx)
    return atom.GetTotalNumHs() > 0


from rdkit.Chem import rdmolfiles
import collections
# Function to average shifts for symmetric carbons
def average_shifts(shift_list, sym_groups):
    avg_shifts = {}
    for sym_group in sym_groups:
        avg_shift = [sum([shift_list.get(i, 0)[0] for i in sym_group]) / len(sym_group)]
        for i in sym_group:
            avg_shifts[i] = avg_shift
    return avg_shifts


# Function to update original shift list with averaged values for symmetric carbons
def update_shifts_with_averaged(original_shifts, averaged_shifts):
    updated_shifts = original_shifts.copy()  # Make a copy of the original dictionary
    for carbon, avg_shift in averaged_shifts.items():
        updated_shifts[carbon] = avg_shift  # Update with the averaged value
    return updated_shifts


# Updated function to make plotting optional and return coordinates of plotted points
import matplotlib.pyplot as plt
# Updated function to avoid duplicate peaks in the plot
def plot_and_save_cosy_spectrum_with_zoom_no_duplicates(heavy_atom_hydrogen_shift_dict, carbon_dict, chiral_centers, plot=True, xlim=None, ylim=None):
    plotted_points = set()  # Using a set to automatically handle duplicates
    
    for carbon1, neighbors in carbon_dict.items():
        h1_shifts = heavy_atom_hydrogen_shift_dict.get(carbon1, [])
        if h1_shifts:
            plotted_points.add((h1_shifts[0], h1_shifts[0]))
            
        for carbon2 in neighbors:
            h2_shifts = heavy_atom_hydrogen_shift_dict.get(carbon2, [])
            if h1_shifts and h2_shifts:
                is_chiral1 = carbon1 in chiral_centers
                is_chiral2 = carbon2 in chiral_centers

                if is_chiral1 or is_chiral2:
                    for h1_shift in h1_shifts:
                        for h2_shift in h2_shifts:
                            plotted_points.add((h1_shift, h2_shift))
                            plotted_points.add((h2_shift, h1_shift))
                else:
                    plotted_points.add((h1_shifts[0], h2_shifts[0]))
                    plotted_points.add((h2_shifts[0], h1_shifts[0]))
                
    x_coords = [x for x, y in plotted_points]
    y_coords = [y for x, y in plotted_points]
    
    if plot:
        plt.scatter(x_coords, y_coords, c='blue', marker='o', label='Cross Peaks', alpha=0.5)
        plt.title('COSY Spectrum with All Diagonal Peaks')
        plt.xlabel('Chemical Shift (ppm)')
        plt.ylabel('Chemical Shift (ppm)')
        plt.grid(True)
        plt.gca().invert_xaxis()
        plt.gca().invert_yaxis()
        plt.legend()
        # Apply zooming window if provided
        if xlim:
            plt.xlim(xlim)
        if ylim:
            plt.ylim(ylim)
        plt.show()
    
    return list(plotted_points)  # Converting set back to list

In [47]:
from IPython.display import display
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG, display
import os
from tqdm import tqdm
from rdkit.Chem.EnumerateStereoisomers import GetStereoisomerCount,EnumerateStereoisomers


def run_COSY_generation(config):
    folder_path = config.SGNN_gen_folder_path
    nmr_files = glob.glob(folder_path+"/*")
    nmr_files = [file for file in nmr_files if "NMR_" in file.split("/")[-1] ]
    nmr_files = [file for file in nmr_files if not ".mol" in file]
    nmr_files = sorted(nmr_files, reverse=True)

    ### Stores
    smiles_list = []
    data_list = []
    sample_id_list = []

    for file_path in tqdm(nmr_files[:]):
        try:
            mol = SDMolSupplier(file_path)[0]
            isomers = tuple(EnumerateStereoisomers(mol))
            stereo_smi = Chem.MolToSmiles(isomers[0],isomericSmiles=True)

            # Get NMR shifts
            averaged_nmr_shifts = mol.GetProp('averaged_NMR_shifts')
            sample_shifts = list(map(float, averaged_nmr_shifts.split()))

            # Extract the sample_id from the path
            file_name = os.path.basename(file_path)
            sample_id = os.path.splitext(file_name)[0].split('NMR_')[-1]

            # Find chiral centers
            chiral_centers = find_chiral_centers(mol)

            carbon_dict = find_carbons_with_relevant_neighbors(mol)

            heavy_atom_dict = find_heavy_atoms_with_hydrogens(mol)

            heavy_atom_hydrogen_shift_dict = extract_symmetric_hydrogen_shifts(sample_shifts, heavy_atom_dict)

            sym_dupl_lists = find_symmetric_positions(stereo_smi)

            # Remove symmetric positions that don't have hydrogens
            sym_dupl_lists = [positions for positions in sym_dupl_lists if all(has_hydrogens(mol, idx) for idx in positions)]

            averaged_shifts = average_shifts(heavy_atom_hydrogen_shift_dict, sym_dupl_lists)

            updated_heavy_atom_hydrogen_shift_dict = update_shifts_with_averaged(heavy_atom_hydrogen_shift_dict, averaged_shifts)

            COSY_shifts = plot_and_save_cosy_spectrum_with_zoom_no_duplicates(heavy_atom_hydrogen_shift_dict, carbon_dict, chiral_centers, plot=False, xlim=None, ylim=None)

            COSY_shifts = sorted(COSY_shifts, key=lambda x: x[0])

            mol = Chem.RemoveHs(mol)
            smi = MolToSmiles(mol)     

            smiles_list.append(smi)
            data_list.append(COSY_shifts)
            sample_id_list.append(sample_id)
        except:
            print(smi)

    # Create a DataFrame with the lists as columns
    data = pd.DataFrame({
        'SMILES': smiles_list,
        'shifts': data_list,
        'sample-id': sample_id_list,
    })

    data.reset_index(drop=True, inplace=True)
    csv_COSY_path = os.path.join(config.SGNN_csv_save_folder, "data_COSY.csv")
    data.to_csv(csv_COSY_path, index=False)
    return data




### HSQC functions

In [52]:
import utils.nmr_calculation_from_dft as ncfd


def run_HSQC_generation(config):
    
    folder_path = config.SGNN_gen_folder_path
    nmr_files = glob.glob(folder_path+"/*")
    nmr_files = [file for file in nmr_files if "NMR_" in file.split("/")[-1] ]
    nmr_files = [file for file in nmr_files if not ".mol" in file]
    nmr_files = sorted(nmr_files, reverse=True)
    ### Stores
    smiles_list = []
    data_list = []
    sample_id_list = []


    for file_path in tqdm(nmr_files[:]):
        try:
            sample_df = ncfd.load_dft_dft_comparison(file_path)

            # Extract the sample_id from the path
            file_name = os.path.basename(file_path)
            sample_id = os.path.splitext(file_name)[0].split('NMR_')[-1]

            # Use the apply method to create a list of lists
            HSQC_shifts = sample_df.apply(lambda row: [row['F2 (ppm)'], row['F1 (ppm)']], axis=1).tolist()
            HSQC_shifts = sorted(HSQC_shifts, key=lambda x: x[0])

            mol = SDMolSupplier(file_path)[0]   
            mol = Chem.RemoveHs(mol)
            smi = MolToSmiles(mol)   

            smiles_list.append(smi)
            data_list.append(HSQC_shifts)
            sample_id_list.append(sample_id)
        except:
            print(smi, sample_id)

    # Create a DataFrame with the lists as columns
    data = pd.DataFrame({
        'SMILES': smiles_list,
        'shifts': data_list,
        'sample-id': sample_id_list,
    })

    data.reset_index(drop=True, inplace=True)
    csv_HSQC_path = os.path.join(config.SGNN_csv_save_folder, "data_HSQC.csv")

    data.to_csv(csv_HSQC_path)
    return data

In [53]:
def main_run_data_generation(config):
    combined_df = run_sgnn(config)
    data_1H = run_1H_generation(config)
    data_13C = run_13C_generation(config)
    data_COSY = run_COSY_generation(config)
    data_HSQC = run_HSQC_generation(config)
    return combined_df, data_1H, data_13C, data_COSY, data_HSQC

100%|██████████| 10/10 [00:00<00:00, 71.08it/s]
